In [ ]:
from json import *
file='training'
with open(file,'r')as f:
    gold=f.readlines()
texts=[]
label_gold=[]
sentence=[]
char_ner_label=[]

label_dic={'B-LOC': [1,0,0,0,0,0,0], 'I-LOC': [0,1,0,0,0,0,0],
 'B-ORG': [0,0,1,0,0,0,0], 'I-ORG': [0,0,0,1,0,0,0],
 'B-PER': [0,0,0,0,1,0,0], 'I-PER': [0,0,0,0,0,1,0],
 'N': [0,0,0,0,0,0,1]}
symbol=['。','，',',',';','》','?','!','？','！']
maxlen=0
for i in gold:
    if len(i.split())==2:

        if len(sentence)>90 and i.split()[1]=='N':
            texts.append(sentence)
            label_gold.append(char_ner_label)
            if len(sentence)>maxlen:
                maxlen=len(sentence)
            print(len(sentence))
            sentence = []
            char_ner_label = []
            
        if len(sentence)>90 and (i.split()[1]=='B-LOC' or i.split()[1]=='B-ORG' or i.split()[1]=='B-PER' ):
            texts.append(sentence)
            label_gold.append(char_ner_label)
            if len(sentence)>maxlen:
                maxlen=len(sentence)
            print(len(sentence))
            sentence = []
            char_ner_label = []
        sentence.append(i.split()[0])
        char_ner_label.append(label_dic[i.split()[1]])
    else:
        texts.append(sentence)
        label_gold.append(char_ner_label)
        if len(sentence)>maxlen:
            maxlen=len(sentence)
        print(len(sentence))
        sentence = []
        char_ner_label = []



from pyltp import Segmentor
segmentor = Segmentor()
segmentor.load('cws.model')

seg_labels = []
for i in range(len(texts)):
    words = segmentor.segment(''.join(texts[i]))
    seg_labels_temp = []

    for word in words:

        if len(unicode(word))==1:
            seg_labels_temp += [[1,0,0,0]]
        if len(unicode(word))==2:
            seg_labels_temp += [[0,1,0,0]]
            seg_labels_temp += [[0,0,0,1]]
        if len(unicode(word))>2 :
            seg_labels_temp += [[0,1,0,0]]
            seg_labels_temp += [[0,0,1,0]]*(len(unicode(word))-2)
            seg_labels_temp += [[0,0,0,1]]

    if len(seg_labels_temp)!=len(texts[i]):
        print(list(words))
        print(len(seg_labels_temp),len(texts[i]))
        for i in words:
            print(i)
    assert len(seg_labels_temp)==len(texts[i])
    seg_labels.append(seg_labels_temp)


file='gold8'
with open(file,'r')as f:
    test_gold=f.readlines()
test_texts=[]
test_label_gold=[]
sentence=[]
char_ner_label=[]

maxlen=0

for i in test_gold:

    if len(i.split())==2:
        if len(sentence)>90 and i.split()[1]=='N':
            test_texts.append(sentence)
            test_label_gold.append(char_ner_label)
            if len(sentence)>maxlen:
                maxlen=len(sentence)
            print(len(sentence))
            sentence = []
            char_ner_label = []
            
        if len(sentence)>90 and (i.split()[1]=='B-LOC' or i.split()[1]=='B-ORG' or i.split()[1]=='B-PER' ):
            test_texts.append(sentence)
            test_label_gold.append(char_ner_label)
            if len(sentence)>maxlen:
                maxlen=len(sentence)
            print(len(sentence))
            sentence = []
            char_ner_label = []
        sentence.append(i.split()[0])
        char_ner_label.append(label_dic[i.split()[1]])
    else:
        test_texts.append(sentence)
        test_label_gold.append(char_ner_label)
        if len(sentence)>maxlen:
            maxlen=len(sentence)
        print(len(sentence))
        sentence = []
        char_ner_label = []
        
print(maxlen)

test_seg_labels = []
for i in range(len(test_texts)):
    words = segmentor.segment(''.join(test_texts[i]))
    seg_labels_temp = []

    for word in words:
        #print(len(unicode(word)))
        if len(unicode(word))==1:
            seg_labels_temp += [[1,0,0,0]]
        if len(unicode(word))==2:
            seg_labels_temp += [[0,1,0,0]]
            seg_labels_temp += [[0,0,0,1]]
        if len(unicode(word))>2 :
            seg_labels_temp += [[0,1,0,0]]
            seg_labels_temp += [[0,0,1,0]]*(len(unicode(word))-2)
            seg_labels_temp += [[0,0,0,1]]

    if len(seg_labels_temp)!=len(test_texts[i]):
        print(list(words))
        print(len(seg_labels_temp),len(test_texts[i]))
        for i in words:
            print(i)
    assert len(seg_labels_temp)==len(test_texts[i])
    test_seg_labels.append(seg_labels_temp)



word_dict={'PADDING':0,'UNK':1}

for i in gold:
    if len(i.split())==2:
        sentence.append(i.split()[0])
        if not word_dict.has_key(str(i.split()[0])):
            word_dict[str(i.split()[0])]=len(word_dict)
for i in test_gold:
    if len(i.split())==2:
        sentence.append(i.split()[0])
        if not word_dict.has_key(str(i.split()[0])):
            word_dict[str(i.split()[0])]=len(word_dict)


import numpy as np
embdict={}
plo=0
import cPickle as pickle
with open('embedding.txt','rb')as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    #print(binary_len)
    print(vocab_size,layer1_size)
    for line in range(vocab_size):
        linesplit = f.readline().split()
        #print(line,len(k))
        word=linesplit[0]
        if len(word) != 0:
            vec=[float(x) for x in linesplit[1:]]
            if word_dict.has_key(str(word)):
                embdict[str(word)]=vec


from numpy.linalg import cholesky

charemb=[0]*len(word_dict)
xp=np.zeros(200,dtype='float32')

char_emb_voca=[]
for i in embdict.keys():
    charemb[word_dict[i]]=np.array(embdict[i],dtype='float32')
    char_emb_voca.append(charemb[word_dict[i]])
char_emb_voca=np.array(char_emb_voca,dtype='float32')

mu=np.mean(char_emb_voca, axis=0)
Sigma=np.cov(char_emb_voca.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(charemb)):
    if type(charemb[i])==int:
        charemb[i]=np.reshape(norm, 200)
charemb[0]=np.zeros(200,dtype='float32')
charemb=np.array(charemb,dtype='float32')
print(charemb.shape)

numbict={}
for key in word_dict.keys():
    numbict[word_dict[key]]=str(key)

def same_sample(string,target,name=False):
    temp=target
    random.shuffle(temp)
    if not name:
        for k in temp:
            if k!=string and unicode(k)[-1]==unicode(string)[-1] and len(unicode(string))>1:
                return k
    if name:
        for k in temp:
            if k!=string  and len(unicode(string))>1:
                return k
    return string

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf  
import keras
from keras import activations 
from keras import initializers 
from keras import constraints 
from keras import regularizers
from keras.layers import *
from keras import backend as K
from keras import metrics
from keras.models import Sequential,Model
from keras.activations import *
from keras.initializers import *
from keras.constraints import *
from keras_contrib.layers import CRF

In [ ]:
for time in range(1):
    training_word=[]
    training_label=[]
    training_label_seg=[]
    test_word=[]
    test_label=[]
    
    
    maxlen=110
    exceed=0
    for i in texts:
        char_list=[]
        for j in i:
            char_list.append(word_dict[str(j)])
        char_list+=(maxlen-len(char_list))*[0]
        training_word.append(char_list)
    
    for i in test_texts:
        char_list=[]
        for j in i:
            char_list.append(word_dict[str(j)])
        char_list+=(maxlen-len(char_list))*[0]
        test_word.append(char_list)
        
    for i in label_gold:
        training_label.append(i+[[0,0,0,0,0,0,1]]*(maxlen-len(i)))
        
    for i in test_label_gold:
        test_label.append(i+[[0,0,0,0,0,0,1]]*(maxlen-len(i)))
    
    for i in seg_labels:
        training_label_seg.append(i+[[0,0,0,1]]*(maxlen-len(i)))

In [1]:
for time in range(1):

    import random
    sample_number=int(len(training_word)*0.05)
    random_index=random.sample(range(len(training_word)),sample_number)
    #
    training_word=  np.array(training_word,dtype='float32')   [random_index]
    training_label= np.array(training_label,dtype='float32')  [random_index]
    training_label_seg= np.array(training_label_seg,dtype='float32')[random_index]
    
    test_word=      np.array(test_word,dtype='float32')
    test_label=     np.array(test_label,dtype='float32')
    
    ner_dict={}
    loc_dict={}
    per_dict={}
    org_dict={}
    part=len(training_label)
    ner_idlabel=np.argmax(training_label[:part],axis=2)
    seg_idlabel=np.argmax(training_label_seg[:part],axis=2)
    for i in range(len(ner_idlabel)):
        loc_ls=[]
        org_ls=[]
        per_ls=[]
        temp_label=training_label[i]
        temp_sent=training_word[i]
        loc_temp=[]
        org_temp=[]
        per_temp=[]
        per_begin=-1
        per_end=-1
        loc_begin=-1
        loc_end=-1
        org_begin=-1
        org_end=-1
        for j in range(len(ner_idlabel[i])):
            if ner_idlabel[i][j]==0:
                if len(loc_temp)>0:
                    loc_end=j
                    ner_dict[str(''.join(loc_temp))]=seg_idlabel[i][loc_begin:loc_end]
                    loc_dict[str(''.join(loc_temp))]=len(loc_dict)
                    loc_temp=[]
                if len(org_temp)>0:
                    org_end=j
                    ner_dict[str(''.join(org_temp))]=seg_idlabel[i][org_begin:org_end]
                    org_dict[str(''.join(org_temp))]=len(org_dict)
                    org_temp=[]
                    
                if len(per_temp)>0:
                    per_end=j
                    ner_dict[str(''.join(per_temp))]=seg_idlabel[i][per_begin:per_end]
                    per_dict[str(''.join(per_temp))]=len(per_dict)
                    per_temp=[]
                loc_temp.append(numbict[int(training_word[i][j])])
                loc_begin=j
            if ner_idlabel[i][j]==1  and len(loc_temp)>0:
                loc_temp.append(numbict[int(training_word[i][j])])
            if ner_idlabel[i][j]==6 and len(loc_temp)>0:
                loc_end=j
                ner_dict[str(''.join(loc_temp))]=seg_idlabel[i][loc_begin:loc_end]
                loc_dict[str(''.join(loc_temp))]=len(loc_dict)
                loc_temp=[]
                
            if ner_idlabel[i][j]==2:
                if len(loc_temp)>0:
                    loc_end=j
                    ner_dict[str(''.join(loc_temp))]=seg_idlabel[i][loc_begin:loc_end]
                    loc_dict[str(''.join(loc_temp))]=len(loc_dict)
                    loc_temp=[]
                if len(org_temp)>0:
                    org_end=j
                    ner_dict[str(''.join(org_temp))]=seg_idlabel[i][org_begin:org_end]
                    org_dict[str(''.join(org_temp))]=len(org_dict)
                    org_temp=[]
                if len(per_temp)>0:
                    per_end=j
                    ner_dict[str(''.join(per_temp))]=seg_idlabel[i][per_begin:per_end]
                    per_dict[str(''.join(per_temp))]=len(per_dict)
                    per_temp=[]
                org_temp.append(numbict[int(training_word[i][j])])
                org_begin=j
            if ner_idlabel[i][j]==3  and len(org_temp)>0:
                org_temp.append(numbict[int(training_word[i][j])])
            if ner_idlabel[i][j]==6 and len(org_temp)>0:
                org_end=j
                ner_dict[str(''.join(org_temp))]=seg_idlabel[i][org_begin:org_end]
                org_dict[str(''.join(org_temp))]=len(org_dict)
                org_temp=[]
        
                
            if ner_idlabel[i][j]==4:
                if len(loc_temp)>0:
                    loc_end=j
                    ner_dict[str(''.join(loc_temp))]=seg_idlabel[i][loc_begin:loc_end]
                    loc_dict[str(''.join(loc_temp))]=len(loc_dict)
                    loc_temp=[]
                if len(org_temp)>0:
                    org_end=j
                    ner_dict[str(''.join(org_temp))]=seg_idlabel[i][org_begin:org_end]
                    org_dict[str(''.join(org_temp))]=len(org_dict)
                    org_temp=[]
                if len(per_temp)>0:
                    per_end=j
                    ner_dict[str(''.join(per_temp))]=seg_idlabel[i][per_begin:per_end]
                    per_dict[str(''.join(per_temp))]=len(per_dict)
                    per_temp=[]
                per_temp.append(numbict[int(training_word[i][j])])
                per_begin=j
            if ner_idlabel[i][j]==5  and len(per_temp)>0:
                per_temp.append(numbict[int(training_word[i][j])])
            if ner_idlabel[i][j]==6 and len(per_temp)>0:
                per_end=j
                ner_dict[str(''.join(per_temp))]=seg_idlabel[i][per_begin:per_end]
                per_dict[str(''.join(per_temp))]=len(per_dict)
                per_temp=[]
            #print(per_begin,per_end,loc_begin,loc_end,org_begin,org_end)
            #print(loc_temp,org_temp,per_temp)
    loc_dict=loc_dict.keys()
    per_dict=per_dict.keys()
    org_dict=org_dict.keys()   

    #print((per_dict))
    
    
    import random
    
    ner_idlabel=np.argmax(training_label,axis=2)
    
    new_train_word=[]
    new_train_label=[]
    new_train_label_seg=[]
    enough_sample=False
    while len(new_train_word)<int(len(training_label)*0.9):
        if enough_sample:
            break
        for i in range(len(ner_idlabel)):
            #print('******')
            if len(new_train_word)==len(training_label)*0.9:
                enough_sample=True
                break
            
            loc_ls=[]
            org_ls=[]
            per_ls=[]
            temp_id_label=np.argmax(training_label[i],axis=1).tolist()
            temp_id_label_seg=np.argmax(training_label_seg[i],axis=1).tolist()
            temp_id_sent=training_word[i].tolist()
            
            
            temp_label=[]#np.argmax(training_label[i],axis=1).tolist()
            temp_label_seg=[]#np.argmax(training_label_seg[i],axis=1).tolist()
            temp_label_seg_gold=np.argmax(training_label_seg[i],axis=1).tolist()
            temp_sent=[]#training_word[i].tolist()
            change_dic={}
            loc_temp=[]
            org_temp=[]
            per_temp=[]
            per_begin=-1
            per_end=-1
            loc_begin=-1
            loc_end=-1
            org_begin=-1
            org_end=-1

            reserve=False
            for j in range(len(ner_idlabel[i])):
                if ner_idlabel[i][j]==0:
                    
                    if len(loc_temp)>0:
                        loc_end=j
                        #ner_dict[str(''.join(loc_temp))]=len(ner_dict)
                        loc_ls.append(''.join(loc_temp))
                        if len(loc_temp)>=2:
                            name_rand=unicode(same_sample(''.join(loc_temp),loc_dict))   
                            name_tp_list=[]
                            for r in name_rand:
                                name_tp_list.append(word_dict[str(r)])
                            if name_rand!=''.join(loc_temp):
                                reserve=True    
                            change_dic[loc_begin]=[loc_end,name_tp_list,[0]+[1]*(len(name_tp_list)-1),ner_dict[str(name_rand)].tolist()]
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_end=j
                        #ner_dict[str(''.join(org_temp))]=len(ner_dict)
                        org_ls.append(''.join(org_temp))
                        org_temp=[]
                    if len(per_temp)>0:
                        per_end=j
                        #ner_dict[str(''.join(per_temp))]=len(ner_dict)
                        per_ls.append(''.join(per_temp))
                        per_temp=[]
                    loc_temp.append(numbict[int(training_word[i][j])])
                    loc_begin=j
                    
                if ner_idlabel[i][j]==1  and len(loc_temp)>0:
                    loc_temp.append(numbict[int(training_word[i][j])])
                    
                if ner_idlabel[i][j]==6 and len(loc_temp)>0:
                    loc_end=j
                    #ner_dict[str(''.join(loc_temp))]=len(ner_dict)
                    loc_ls.append(''.join(loc_temp))
                    #print(len(loc_temp))
                    if len(loc_temp)>=2:
                        name_rand=unicode(same_sample(''.join(loc_temp),loc_dict))   
                        name_tp_list=[]
                        for r in name_rand:
                            name_tp_list.append(word_dict[str(r)])
                        if name_rand!=''.join(loc_temp):
                            reserve=True   
                        change_dic[loc_begin]=[loc_end,name_tp_list,[0]+[1]*(len(name_tp_list)-1),ner_dict[str(name_rand)].tolist()]
                    loc_temp=[]
                    
                if ner_idlabel[i][j]==2:
                    if len(loc_temp)>0:
                        loc_end=j
                        #ner_dict[str(''.join(loc_temp))]=len(ner_dict)
                        loc_ls.append(''.join(loc_temp))
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_end=j
                        #ner_dict[str(''.join(org_temp))]=len(ner_dict)
                        org_ls.append(''.join(org_temp))
                        if len(org_temp)>=2:
                            name_rand=unicode(same_sample(''.join(org_temp),org_dict))   
                            name_tp_list=[]
                            for r in name_rand:
                                name_tp_list.append(word_dict[str(r)])
                            if name_rand!=''.join(org_temp):
                                reserve=True   
                            change_dic[org_begin]=[org_end,name_tp_list,[2]+[3]*(len(name_tp_list)-1),ner_dict[str(name_rand)].tolist()]
                        org_temp=[]
                    if len(per_temp)>0:
                        per_end=j
                        #ner_dict[str(''.join(per_temp))]=len(ner_dict)
                        per_ls.append(''.join(per_temp))
                        per_temp=[]
                    org_temp.append(numbict[int(training_word[i][j])])
                    org_begin=j
                if ner_idlabel[i][j]==3  and len(org_temp)>0:
                    org_temp.append(numbict[int(training_word[i][j])])
                if ner_idlabel[i][j]==6 and len(org_temp)>0:
                    org_end=j
                    #ner_dict[''.join(org_temp)]=len(ner_dict)
                    org_ls.append(''.join(org_temp))
                    if len(org_temp)>=2:
                        name_rand=unicode(same_sample(''.join(org_temp),org_dict))   
                        name_tp_list=[]
                        for r in name_rand:
                            name_tp_list.append(word_dict[str(r)])
                        if name_rand!=''.join(org_temp):
                            reserve=True     
                        change_dic[org_begin]=[org_end,name_tp_list,[2]+[3]*(len(name_tp_list)-1),ner_dict[str(name_rand)].tolist()]
                    org_temp=[]
            
                    
                if ner_idlabel[i][j]==4:
                    if len(loc_temp)>0:
                        loc_end=j
                        #ner_dict[str(''.join(loc_temp))]=len(ner_dict)
                        loc_ls.append(''.join(loc_temp))
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_end=j
                        #ner_dict[str(''.join(org_temp))]=len(ner_dict)
                        org_ls.append(''.join(org_temp))
                        org_temp=[]
                    if len(per_temp)>0:
                        per_end=j
                        #ner_dict[str(''.join(per_temp))]=len(ner_dict)
                        per_ls.append(''.join(per_temp))
                        if len(per_temp)>=2:
                            name_rand=unicode(same_sample(''.join(per_temp),per_dict,True))   
                            name_tp_list=[]
                            for r in name_rand:
                                name_tp_list.append(word_dict[str(r)])
                            if name_rand!=''.join(per_temp):
                                reserve=True     
                            change_dic[per_begin]=[per_end,name_tp_list,[4]+[5]*(len(name_tp_list)-1),ner_dict[str(name_rand)].tolist()]
                        per_temp=[]
                    per_temp.append(numbict[int(training_word[i][j])])
                    per_begin=j
                if ner_idlabel[i][j]==5  and len(per_temp)>0:
                    per_temp.append(numbict[int(training_word[i][j])])
                if ner_idlabel[i][j]==6 and len(per_temp)>0:
                    #ner_dict[''.join(per_temp)]=len(ner_dict)
                    per_ls.append(''.join(per_temp))
                    per_end=j
                    if len(per_temp)>=2:
                        name_rand=unicode(same_sample(''.join(per_temp),per_dict,True))   
                        name_tp_list=[]
                        for r in name_rand:
                            name_tp_list.append(word_dict[str(r)])
                        if name_rand!=''.join(per_temp):
                            reserve=True     
                        change_dic[per_begin]=[per_end,name_tp_list,[4]+[5]*(len(name_tp_list)-1),ner_dict[str(name_rand)].tolist()]
                    per_temp=[]
                #print(per_begin,per_end,loc_begin,loc_end,org_begin,org_end)
            sparse_temp_label=[]
            aug_id=0
            while aug_id < len(ner_idlabel[i]):
                if aug_id in change_dic.keys():
                    temp_sent+=change_dic[aug_id][1]
                    temp_label+=change_dic[aug_id][2]
                    temp_label_seg+=change_dic[aug_id][3]
                    aug_id=change_dic[aug_id][0]-1
                else:
                    temp_sent.append(temp_id_sent[aug_id])
                    temp_label.append(temp_id_label[aug_id])
                    temp_label_seg.append(temp_id_label_seg[aug_id])
                aug_id+=1
            for gg in temp_label:
                tpsp=[0,0,0,0,0,0,0]
                tpsp[gg]=1
                sparse_temp_label.append(tpsp)
                
            sparse_temp_label_seg=[]
            for gg in temp_label_seg:
                tpsp=[0,0,0,0]
                tpsp[gg]=1
                sparse_temp_label_seg.append(tpsp)
            #print(len(temp_label_seg))
            if len(temp_sent)<=120 and reserve:
                if not len(temp_sent)==len(temp_label)==len(temp_label_seg):
                    print(len(temp_sent),len(temp_label),len(temp_label_seg))
                    assert False
                

                print(''.join([numbict[a] for a in temp_sent]))
                print(temp_label)
                print(''.join([numbict[a] for a in training_word[i]]))
                print(ner_idlabel[i])
                new_train_word.append(temp_sent) 
                new_train_label.append(sparse_temp_label)
                new_train_label_seg.append(sparse_temp_label_seg)
                
    training_word=[]
    training_label=[]
    training_label_seg=[]
    test_word=[]
    test_label=[]
    
    
    maxlen=120
    exceed=0
    for i in texts:
        tp=[]
        for j in i:
            tp.append(word_dict[str(j)])
        tp+=(maxlen-len(tp))*[0]
        training_word.append(tp)
    
    for i in test_texts:
        tp=[]
        for j in i:
            tp.append(word_dict[str(j)])
        tp+=(maxlen-len(tp))*[0]
        test_word.append(tp)
        
    for i in label_gold:
        training_label.append(i+[[0,0,0,0,0,0,1]]*(maxlen-len(i)))
        
    for i in test_label_gold:
        test_label.append(i+[[0,0,0,0,0,0,1]]*(maxlen-len(i)))
    
    for i in seg_labels:
        training_label_seg.append(i+[[0,0,0,1]]*(maxlen-len(i)))
        
    maxlen=120
    newer_train_word=[]
    newer_train_label=[]
    newer_train_label_seg=[]
    for i in new_train_word:
        newer_train_word.append(i+(maxlen-len(i))*[0])
        
    for i in new_train_label:
        newer_train_label.append(i+[[0,0,0,0,0,0,1]]*(maxlen-len(i)))
    
    for i in new_train_label_seg:
        newer_train_label_seg.append(i+[[0,0,0,1]]*(maxlen-len(i)))
        
    valid_word=  np.array(training_word,dtype='float32')   [random_index][int(sample_number*0.9):]
    valid_label= np.array(training_label,dtype='float32')  [random_index][int(sample_number*0.9):]
    valid_label_seg= np.array(training_label_seg,dtype='float32')[random_index][int(sample_number*0.9):]
    test_word=      np.array(test_word,dtype='float32')
    test_label=     np.array(test_label,dtype='float32') 
    task=[('uselstmsoft',True,False,'none'), ('uselstm',False,False,'none'),
          ('uselstm',True,False,'none'),('uselstm',True,False,'peng'),('uselstm',True,True,'none'),
          ('usecnnlstmsoft',True,False,'none'),('usecnnlstm',False,False,'none'),('usecnnlstm',True,False,'none'),
          ('usecnnlstm',True,False,'joint'),('usecnnlstm',True,True,'none'),('usecnnlstm',False,True,'joint'),('usecnnlstm',True,True,'joint')]

    for phase in range(4,5):   
        loc_result=[]
        org_result=[]
        per_result=[]
        total_result=[]
        oov_result=[]
        archi=task[phase][0]
        hasemb=task[phase][1]
        pseudo=task[phase][2]
        joint=task[phase][3]
        pseudostr=' '
        if not pseudo:       
            training_word=  np.concatenate([np.array(training_word,dtype='float32')   [random_index][:int(sample_number*0.9)]])
            training_label= np.concatenate([np.array(training_label,dtype='float32')  [random_index][:int(sample_number*0.9)]])
            training_label_seg= np.concatenate([np.array(training_label_seg,dtype='float32')[random_index][:int(sample_number*0.9)]])
        if pseudo: 
            pseudostr='+pseudo'
            training_word=  np.concatenate([np.array(training_word,dtype='float32')   [random_index][:int(sample_number*0.9)],np.array(newer_train_word)])
            training_label= np.concatenate([np.array(training_label,dtype='float32')  [random_index][:int(sample_number*0.9)],np.array(newer_train_label)])
            training_label_seg= np.concatenate([np.array(training_label_seg,dtype='float32')[random_index][:int(sample_number*0.9)],np.array(newer_train_label_seg)])

        sequenceLength =120
        fmin=0
        batch_size=64
        x_train = Input(batch_shape=(None,sequenceLength),dtype='float32')
        
        emb_drop=0.2
        global_drop=0.2
        hid1=200
        model0 = Sequential()
        embstr=' '
        if not hasemb:
            embstr='w/o emb'
            model0.add(Embedding(len(word_dict), 200,input_length=sequenceLength,mask_zero=False))
        else:
            model0.add(Embedding(len(word_dict), 200,weights=[charemb],input_length=sequenceLength,mask_zero=False))
    
        model0.add(Dropout(emb_drop))
        
        
        
        e0=model0(x_train)
        if archi=='usecnn':
            conv2 = Convolution1D(nb_filter=100, filter_length=2, border_mode='same', activation='relu')(e0)
            conv3 = Convolution1D(nb_filter=100, filter_length=3, border_mode='same', activation='relu')(e0)
            conv4 = Convolution1D(nb_filter=100, filter_length=4, border_mode='same', activation='relu')(e0)
            conv5 = Convolution1D(nb_filter=100, filter_length=5, border_mode='same', activation='relu')(e0)
            merge = concatenate([conv2, conv3, conv4, conv5])
            merge = Dropout(0.2)(merge)
            crf = CRF(7,sparse_target=False)
            cr=crf(merge)
            
            
        if archi=='uselstm':
            blstm1 = Bidirectional(LSTM(200, return_sequences=True), merge_mode='concat')(e0)
            blstm1 = Dropout(0.2)(blstm1)
            crf = CRF(7,sparse_target=False)
            cr=crf(blstm1)
            
        if archi=='usecnnlstm':
            conv2 = Convolution1D(nb_filter=100, filter_length=2, border_mode='same', activation='relu')(e0)
            conv3 = Convolution1D(nb_filter=100, filter_length=3, border_mode='same', activation='relu')(e0)
            conv4 = Convolution1D(nb_filter=100, filter_length=4, border_mode='same', activation='relu')(e0)
            conv5 = Convolution1D(nb_filter=100, filter_length=5, border_mode='same', activation='relu')(e0)
            merge = concatenate([conv2, conv3, conv4, conv5])
            merge = Dropout(0.2)(merge)
    
            blstm1 = Bidirectional(LSTM(200, return_sequences=True), merge_mode='concat')(merge)
            blstm1 = Dropout(0.2)(blstm1)
            crf = CRF(7,sparse_target=False)
            cr=crf(blstm1)
        if joint=='joint':
            crf2 = CRF(4,sparse_target=False)
            cr2=crf2(merge)
            model=Model([x_train],[cr,cr2])
            model.compile('rmsprop', loss=[crf.loss_function,crf2.loss_function], metrics=[crf.accuracy,crf2.accuracy])
            model.fit([training_word],[training_label,training_label_seg],
                          batch_size=batch_size, nb_epoch=30,shuffle=True,validation_data=([valid_word],[valid_label,valid_label_seg]),callbacks=[EarlyStopping(monitor='val_viterbi_acc', patience=5)])
            classes=model.predict([test_word])[0]
    
        if joint=='none':
    
            model=Model([x_train],[cr])
            model.compile('rmsprop', loss=[crf.loss_function], metrics=[crf.accuracy])
            model.fit([training_word],[training_label],
                          batch_size=batch_size, nb_epoch=30,shuffle=True,validation_data=([valid_word],[valid_label]),callbacks=[EarlyStopping(monitor='val_viterbi_acc', patience=5)])
            classes=model.predict([test_word])
            
        if joint=='peng':
            crf2 = CRF(4,sparse_target=False)
            cr2=crf2(blstm1)
            model=Model([x_train],[cr,cr2])
            model.compile('rmsprop', loss=[crf.loss_function,crf2.loss_function], metrics=[crf.accuracy,crf2.accuracy])
            model.fit([training_word],[training_label,training_label_seg],
                          batch_size=batch_size, nb_epoch=30,shuffle=True,validation_data=([valid_word],[valid_label,valid_label_seg]),callbacks=[EarlyStopping(monitor='val_viterbi_acc', patience=5)])
            classes=model.predict([test_word])[0]
    
        part=len(test_label)
        loc_true_num=0.
        org_true_num=0.
        per_true_num=0.
        loc_pred_num=0.
        org_pred_num=0.
        per_pred_num=0.
        loc_matc_num=0.
        org_matc_num=0.
        per_matc_num=0.
        oov_num=0.
        oov_recall_num=0.
        
        loc_oov_num=0.
        org_oov_num=0.
        per_oov_num=0.
        loc_oov_recall_num=0.
        org_oov_recall_num=0.
        per_oov_recall_num=0.
        pred=np.argmax(classes[:part],axis=2)
        true=np.argmax(test_label[:part],axis=2)
        for i in range(part):
            loc_pred=[]
            org_pred=[]
            per_pred=[]
            loc_temp=[]
            org_temp=[]
            per_temp=[]
            for j in range(len(test_word[i])):
                if pred[i][j]==0:
                    if len(loc_temp)>0:
                        loc_pred.append(str(j)+'_'+''.join(loc_temp))
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_pred.append(str(j)+'_'+''.join(org_temp))
                        org_temp=[]    
                    if len(per_temp)>0:
                        per_pred.append(str(j)+'_'+''.join(per_temp))
                        per_temp=[]
                    loc_temp.append(numbict[int(test_word[i][j])])
                if pred[i][j]==1  and len(loc_temp)>0:
                    loc_temp.append(numbict[int(test_word[i][j])])
                if pred[i][j]==6 and len(loc_temp)>0:
                    loc_pred.append(str(j)+'_'+''.join(loc_temp))
                    loc_temp=[]
                    
                if pred[i][j]==2:
                    if len(loc_temp)>0:
                        loc_pred.append(str(j)+'_'+''.join(loc_temp))
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_pred.append(str(j)+'_'+''.join(org_temp))
                        org_temp=[]    
                    if len(per_temp)>0:
                        per_pred.append(str(j)+'_'+''.join(per_temp))
                        per_temp=[]
                    org_temp.append(numbict[int(test_word[i][j])])
                if pred[i][j]==3  and len(org_temp)>0:
                    org_temp.append(numbict[int(test_word[i][j])])
                if pred[i][j]==6 and len(org_temp)>0:
                    org_pred.append(str(j)+'_'+''.join(org_temp))
                    org_temp=[]
        
                    
                if pred[i][j]==4:
                    if len(loc_temp)>0:
                        loc_pred.append(str(j)+'_'+''.join(loc_temp))
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_pred.append(str(j)+'_'+''.join(org_temp))
                        org_temp=[]    
                    if len(per_temp)>0:
                        per_pred.append(str(j)+'_'+''.join(per_temp))
                        per_temp=[]
                    per_temp.append(numbict[int(test_word[i][j])])
                if pred[i][j]==5  and len(per_temp)>0:
                    per_temp.append(numbict[int(test_word[i][j])])
                if pred[i][j]==6 and len(per_temp)>0:
                    per_pred.append(str(j)+'_'+''.join(per_temp))
                    per_temp=[]
                    
                    
                    
            loc_true=[]
            org_true=[]
            per_true=[]
            loc_temp=[]
            org_temp=[]
            per_temp=[]
            for j in range(len(test_word[i])):
                if true[i][j]==0:
                    if len(loc_temp)>0:
                        loc_true.append(str(j)+'_'+''.join(loc_temp))
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_true.append(str(j)+'_'+''.join(org_temp))
                        org_temp=[]
                    if len(per_temp)>0:
                        per_true.append(str(j)+'_'+''.join(per_temp))
                        per_temp=[]
                    loc_temp.append(numbict[int(test_word[i][j])])
                if true[i][j]==1  and len(loc_temp)>0:
                    loc_temp.append(numbict[int(test_word[i][j])])
                if true[i][j]==6 and len(loc_temp)>0:
                    loc_true.append(str(j)+'_'+''.join(loc_temp))
                    loc_temp=[]
                    
                if true[i][j]==2:
                    if len(loc_temp)>0:
                        loc_true.append(str(j)+'_'+''.join(loc_temp))
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_true.append(str(j)+'_'+''.join(org_temp))
                        org_temp=[]
                    if len(per_temp)>0:
                        per_true.append(str(j)+'_'+''.join(per_temp))
                        per_temp=[]
                    org_temp.append(numbict[int(test_word[i][j])])
                if true[i][j]==3  and len(org_temp)>0:
                    org_temp.append(numbict[int(test_word[i][j])])
                if true[i][j]==6 and len(org_temp)>0:
                    org_true.append(str(j)+'_'+''.join(org_temp))
                    org_temp=[]
                 
                    
                if true[i][j]==4:
                    if len(loc_temp)>0:
                        loc_true.append(str(j)+'_'+''.join(loc_temp))
                        loc_temp=[]
                    if len(org_temp)>0:
                        org_true.append(str(j)+'_'+''.join(org_temp))
                        org_temp=[]
                    if len(per_temp)>0:
                        per_true.append(str(j)+'_'+''.join(per_temp))
                        per_temp=[]
                    per_temp.append(numbict[int(test_word[i][j])])
                if true[i][j]==5  and len(per_temp)>0:
                    per_temp.append(numbict[int(test_word[i][j])])
                if true[i][j]==6 and len(per_temp)>0:
                    per_true.append(str(j)+'_'+''.join(per_temp))
                    per_temp=[]
        
                    
            loc_true_num+=len(loc_true)
            org_true_num+=len(org_true)
            per_true_num+=len(per_true)
            
            loc_pred_num+=len(loc_pred)
            org_pred_num+=len(org_pred)
            per_pred_num+=len(per_pred)
        
            for k in loc_true:
                if not ner_dict.has_key(str(k.split('_')[1])):
                    oov_num+=1
                    loc_oov_num+=1
                if k in loc_pred:
                    loc_matc_num+=1
                    if not ner_dict.has_key(str(k.split('_')[1])):
                        oov_recall_num+=1
                        loc_oov_recall_num+=1
                    
            for k in org_true:
                if not ner_dict.has_key(str(k.split('_')[1])):
                    oov_num+=1
                    org_oov_num+=1
                if k in org_pred:
                    org_matc_num+=1 
                    if not ner_dict.has_key(str(k.split('_')[1])):
                        oov_recall_num+=1
                        org_oov_recall_num+=1
                    
            for k in per_true:
                if not ner_dict.has_key(str(k.split('_')[1])):
                    oov_num+=1
                    per_oov_num+=1
                if k in per_pred:
                    per_matc_num+=1
                    if not ner_dict.has_key(str(k.split('_')[1])):
                        oov_recall_num+=1
                        per_oov_recall_num+=1
        
        locp=loc_matc_num/loc_pred_num
        locr=loc_matc_num/loc_true_num
        locf=2*locp*locr/(locp+locr)
        
        orgp=org_matc_num/org_pred_num
        orgr=org_matc_num/org_true_num
        orgf=2*orgp*orgr/(orgp+orgr)
        
        perp=per_matc_num/per_pred_num
        perr=per_matc_num/per_true_num
        perf=2*perp*perr/(perp+perr)
        all_p=(loc_matc_num+org_matc_num+per_matc_num)/(loc_pred_num+org_pred_num+per_pred_num)
        all_r=(loc_matc_num+org_matc_num+per_matc_num)/(loc_true_num+org_true_num+per_true_num)
        all_f=2*all_p*all_r/(all_p+all_r)
        
        print(locp,locr,locf)
        print(orgp,orgr,orgf)
        print(perp,perr,perf)
        print(all_p,all_r,all_f)
        print(oov_recall_num/oov_num)
        loc_result.append([locp,locr,locf])
        org_result.append([orgp,orgr,orgf])
        per_result.append([perp,perr,perf])
        total_result.append([all_p,all_r,all_f,oov_recall_num/oov_num])
        oov_result.append([loc_oov_recall_num/loc_oov_num,org_oov_recall_num/org_oov_num,per_oov_recall_num/per_oov_num])